## Generate and use test trade data

Here we use the RTS 2 Annex 3 model to generate test trade data which we then classify.

In these first examples we generate a couple of really simple cases.  Read the comments in the code which explain what  is happening at each step.

Do run the steps in order.  The later steps build on work done by the earlier steps ... if you skip a step something may break.  If that happens, just start again from the top.

In [1]:
# Import the RTS 2 Annex III taxonomy 
# ... and get hold of the object which represents the root of the taxonomy tree.
import rts2_annex3
root = rts2_annex3.class_root

# Now we print the root object to see what it is.
print(root)

In [2]:
# Now we'll use the taxonomy to generate some test data.
# First a really simple example using a Sub-asset class which has no criteria:
sub_asset_class = root.sub_asset_class_by_name("Exchange Traded Notes (ETNs)")
sample_trade = sub_asset_class.make_test_samples(1)[0]
classification = root.classification_for(subject=sample_trade)
print(classification.classification_dict())

OrderedDict([('RTS2 version', 'EU 2017/583 of 14 July 2016'), ('Asset class', 'Bonds (ETC and ETN bond types)'), ('Sub-asset class', 'Exchange Traded Notes (ETNs)')])


In [3]:
# The model is implemented in Python and the printed result above is just the
# literal form of a Python dictionary.

# We can make the classification easier to understand using JSON.
# ... but note that the content is just the same as above.

import json
print(json.dumps(classification.classification_dict(), indent=4))


{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Notes (ETNs)"
}


In [4]:
# Above we made a sample trade for a specific Sub-asset class.  Now we'll 
# make a sample for a specifc Asset class.  The Asset class will make the 
# sample trade using one (a random one) of it's Sub-asset classes.
# Run this code many times and you'll see that you get about 50/50 of each
# of the 2 (in this case) Sub-asset classes.

asset_class = root.asset_class_by_name("Bonds (ETC and ETN bond types)")
sample = asset_class.make_test_samples(1)[0]
classification = root.classification_for(subject=sample)
print(json.dumps(classification.classification_dict(), indent=4))


{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Bonds (ETC and ETN bond types)",
    "Sub-asset class": "Exchange Traded Commodities (ETCs)"
}


In [5]:
# Now to make a more complex Sub-asset class.  One with lots of criteria
sub_asset_class = root.sub_asset_class_by_name("Energy commodity swaps")
sample = sub_asset_class.make_test_samples(1)[0]
print(sample.__dict__)
classification = root.classification_for(subject=sample)
print(json.dumps(classification.classification_dict(), indent=4))

{'energy_type': 'INRG', 'to_date': datetime.date(2019, 4, 19), 'from_date': datetime.date(2018, 4, 19), 'underlying_energy': 'underlying_energy.value', 'settlement_type': 'settlement_type.value', 'load_type': 'load_type.value', 'asset_class_name': 'Commodity Derivatives', 'sub_asset_class_name': 'Energy commodity swaps', 'notional_currency': 'notional_currency.value', 'delivery': 'delivery.value'}
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Commodity Derivatives",
    "Sub-asset class": "Energy commodity swaps",
    "Segmentation criterion 1 description": "energy type: oil, oil distillates, coal, oil light ends, natural gas, electricity, inter-energy",
    "Segmentation criterion 1": "INRG",
    "Segmentation criterion 2 description": "underlying energy",
    "Segmentation criterion 2": "underlying_energy.value",
    "Segmentation criterion 3 description": "notional currency defined as the currency in which the notional amount of the swap is denominated",
 

In [6]:
# Generate a sample trade for each Sub-asset class in the taxonomy
samples = []
for sub_asset_class in root.all_sub_asset_classes():
    samples.extend(sub_asset_class.make_test_samples(1))

# Classify each sample
classifications = [root.classification_for(subject=sample) for sample in samples]

# Show a couple of random examples of the classifications
import random
print(json.dumps(random.choice(classifications).classification_dict(), indent=4))
print(json.dumps(random.choice(classifications).classification_dict(), indent=4))


{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Emission Allowance Derivatives",
    "Sub-asset class": "Emission allowance derivatives whose underlying is of the type European Union Allowances (EUA)"
}
{
    "RTS2 version": "EU 2017/583 of 14 July 2016",
    "Asset class": "Interest Rate Derivatives",
    "Sub-asset class": "IR futures and FRA",
    "Segmentation criterion 1 description": "underlying interest rate",
    "Segmentation criterion 1": "underlying_interest_rate.value",
    "Segmentation criterion 2 description": "term of the underlying interest rate",
    "Segmentation criterion 2": "term_of_underlying_interest_rate.value",
    "Segmentation criterion 3 description": "time to maturity bucket of the swap defined as follows:",
    "Segmentation criterion 3": "Maturity bucket 1: Zero to 3 months"
}
